In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import gensim

from wordcloud import WordCloud
from gensim import corpora
from gensim import models
from gensim.corpora.dictionary import Dictionary

from collections import defaultdict
from gensim.parsing.preprocessing import STOPWORDS

In [2]:
import string
import logging

In [3]:
dfname = 'FP4'

df1 = pd.read_pickle('dfs/' + dfname)
df = df1[['title','objective']]
df = df.dropna(how='any')
df['merged'] = df['title'] + ' ' + df['objective']

In [4]:
objectives = df['merged']
# objectives = pd.concat([ df['objective'] for df in [df4, df5, df6, df7, df20] ])

In [5]:
RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])

objectives = objectives.str.lower().str.replace('%l', '').str.replace(RE_PUNCTUATION, ' ')

In [6]:
objectives_split = objectives.str.strip().str.split()
objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if len(token) > 2])
objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if not(token.isdigit())])
objectives_split.head(2)

0                                       [spot, tation]
1    [formation, and, occurrence, nitrous, acd, the...
Name: merged, dtype: object

In [7]:
list_stopwords = ['data','will', 'develop', 'development', 'project', 'research', 'new', 'use', 'europe', 'european', 'based']
if dfname == 'FP4':
    list_stopwords.append('des')

additional_stopwords = set(list_stopwords)
stopwords = set(STOPWORDS) | additional_stopwords

objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if token not in stopwords])
objectives_split.head(2)

0                                       [spot, tation]
1    [formation, occurrence, nitrous, acd, atmosphe...
Name: merged, dtype: object

In [8]:
frequency = defaultdict(int)
for text in objectives_split:
    for token in text:
        frequency[token] += 1

objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if (frequency[token] > 5)])

objectives_dictionary = Dictionary(objectives_split)

In [9]:
class ObjectivesCorpus(corpora.textcorpus.TextCorpus):
    def get_texts(self):
        return iter(self.input)
    def __calc_corpus_size__(self):
        logging.info('Calculating corpus size')
        self.length = 0
        self.num_words = 0
        for doc in self.get_texts():
            self.length += 1
            self.num_words += len(doc)
    def __len__(self):
        """Define this so we can use `len(corpus)`"""
        if 'length' not in self.__dict__:
            self.__calc_corpus_size__()
        return self.length
    def __str__(self):
        if 'num_words' not in self.__dict__:
            self.__calc_corpus_size__()
        return (str(self.length) + ' documents, ' + str(self.num_words)
                + ' words')
            

objectives_corpus = ObjectivesCorpus(objectives_split)

In [10]:
lda = gensim.models.ldamodel.LdaModel(corpus = objectives_corpus, 
                                        id2word = objectives_dictionary, 
                                        num_topics = 10,
                                        passes=2,
                                        random_state = np.random.seed(12),
                                        iterations = 7000)

In [11]:
lda.show_topics()

[(0,
  '0.024*"energy" + 0.014*"systems" + 0.011*"power" + 0.011*"control" + 0.010*"low" + 0.009*"technology" + 0.008*"processing" + 0.008*"design" + 0.008*"integrated" + 0.007*"heat"'),
 (1,
  '0.022*"fish" + 0.014*"cell" + 0.013*"disease" + 0.011*"study" + 0.010*"role" + 0.009*"protein" + 0.008*"growth" + 0.008*"diseases" + 0.008*"cells" + 0.008*"molecular"'),
 (2,
  '0.010*"test" + 0.009*"methods" + 0.007*"testing" + 0.007*"products" + 0.007*"method" + 0.007*"design" + 0.006*"industrial" + 0.006*"materials" + 0.006*"control" + 0.006*"tests"'),
 (3,
  '0.021*"high" + 0.021*"water" + 0.020*"production" + 0.012*"chemical" + 0.012*"food" + 0.010*"materials" + 0.010*"control" + 0.010*"solar" + 0.010*"optical" + 0.009*"quality"'),
 (4,
  '0.027*"food" + 0.017*"health" + 0.012*"industry" + 0.010*"action" + 0.010*"packaging" + 0.009*"technology" + 0.008*"countries" + 0.008*"network" + 0.008*"concerted" + 0.007*"wood"'),
 (5,
  '0.013*"models" + 0.010*"analysis" + 0.008*"innovation" + 0.008*

In [18]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [19]:

pyLDAvis.gensim.prepare(lda, objectives_corpus, objectives_dictionary)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      32.025341        1       1  0.111455  0.159129
7      12.047799        1       2  0.162874 -0.065439
2       9.454423        1       3  0.097768 -0.029428
0       8.243626        1       4  0.132170 -0.093507
5       7.428622        1       5  0.003362  0.145840
1       7.241135        1       6 -0.270171  0.021442
3       6.722546        1       7  0.012893 -0.211867
4       5.921537        1       8  0.026719  0.202619
6       5.459348        1       9 -0.260440 -0.035426
8       5.455624        1      10 -0.016630 -0.093364, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
854   Default  1089.000000           food  1089.000000  30.0000  30.0000
1728  Default  1009.000000         energy  1009.000000  29.0000  29.0000
772   Default   604.000000           fish   604.000000  28.0000  28.0000
9     Default  1703.000000     production  1703.000000  27.0000  27.0000
11    Default  1560.000000           high  1560.000000  26.0000  26.0000
483   Default   767.000000          water   767.000000  25.0000  25.0000
858   Default   756.000000       software   756.000000  24.0000  24.0000
2426  Default   409.000000        genetic   409.000000  23.0000  23.0000
620   Default   549.000000         health   549.000000  22.0000  22.0000
8     Default  1321.000000        process  1321.000000  21.0000  21.0000
86    Default   632.000000          plant   632.000000  20.0000  20.0000
895   Default  1615.000000     technology  1615.000000  19.0000  19.0000
377   Default  1569.000000    information  1569.000000  18.0000  18.0000
1623  Default   539.000000        disease   539.000000  17.0000  17.0000
1611  Default   395.000000           cell   395.000000  16.0000  16.0000
1602  Default   456.000000      molecular   456.000000  15.0000  15.0000
116   Default  1460.000000        quality  1460.000000  14.0000  14.0000
2359  Default   322.000000          genes   322.000000  13.0000  13.0000
711   Default   467.000000         models   467.000000  12.0000  12.0000
139   Default   899.000000       industry   899.000000  11.0000  11.0000
261   Default  1425.000000        network  1425.000000  10.0000  10.0000
736   Default   371.000000     resistance   371.000000   9.0000   9.0000
2702  Default   406.000000        species   406.000000   8.0000   8.0000
587   Default  1097.000000         design  1097.000000   7.0000   7.0000
3424  Default   400.000000            oil   400.000000   6.0000   6.0000
597   Default   459.000000            gas   459.000000   5.0000   5.0000
602   Default   509.000000          power   509.000000   4.0000   4.0000
167   Default   953.000000          study   953.000000   3.0000   3.0000
1548  Default   575.000000      materials   575.000000   2.0000   2.0000
117   Default  1643.000000        control  1643.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
2904  Topic10    56.538909        enzymes    58.313632   2.8776  -5.8622
3547  Topic10    47.816096     wastewater    49.361201   2.8767  -6.0297
3003  Topic10    58.178658       residues    60.703263   2.8660  -5.8336
3648  Topic10    50.352756     harvesting    53.198597   2.8535  -5.9781
1681  Topic10   100.925290     combustion   113.081124   2.7948  -5.2827
2906  Topic10    37.779678          wells    39.121606   2.8736  -6.2653
4274  Topic10    38.299504          blood    39.737477   2.8717  -6.2517
2974  Topic10   175.685475   agricultural   230.567208   2.6367  -4.7284
1717  Topic10    47.623304    nutritional    51.162532   2.8368  -6.0338
604   Topic10    62.176758      injection    70.096934   2.7886  -5.7671
86    Topic10   323.884299          plant   632.520194   2.2392  -4.1167
3424  Topic10   222.695600            oil   400.590033   2.3214  -4.4913
7     Topic10    83.7961